## Configs

This code converts a Middle-format dataset to a ConKADI-format dataset. So before runinning this code, please prepare a Middle-format dataset.

All you need to do is configuring the following 3 terms. Due to the randomness, the generated dataset may have some monior difference.

In [5]:
# CCM source file path

For Chinese Weibo
# See the Step 1
ccm_path = 'ccm/'
# Traget Folder
target_folder = 'base_data/ConKADI/'
vocab_size = 50000
# See the Step 1
file_map = {
    'train':'trainset.txt',
    'dev':'validset.txt',
    'test':'testset.txt',
     'demo':'testset.txt'
}

# For English Reddit

# ccm_path = 'ccm_py/data/'
# target_folder = 'base_data/ConKADI_EN/'
# vocab_size = 30000


# file_map = {
#     'train':'trainset_small.txt',
#     'dev':'validset_small.txt',
#     'test':'testset_small.txt',
#      'demo':'testset_small.txt'
# }


SyntaxError: invalid syntax (<ipython-input-5-0ef69bbb209b>, line 3)

# Format check

In [14]:
import json
data_train = []
with open(ccm_path + file_map['train']) as f:
    for idx, line in enumerate(f):
        data_train.append(json.loads(line))
        if len(data_train) % 1000 == 0: 
            break
            
with open(ccm_path + 'resource.txt') as f:
        d = json.loads(f.readline())
        csk_triples = d['csk_triples']
        csk_entities = d['csk_entities']
        raw_vocab = d['vocab_dict']
        kb_dict = d['dict_csk']

entity_vocab = set()
with open(ccm_path + 'entity.txt',encoding='utf-8') as f:
    for idx, line in enumerate(f):
        entity = line.strip('\n')
        entity_vocab.add(entity)
        
word_vocab = set(raw_vocab)
entity_full_vocab = set(csk_entities)
print('#Word:%d  #Entity:%d' % (len(word_vocab), len(entity_vocab)))

for my_case in data_train:
    flag = False
    for word,idx in zip(my_case['response'],my_case['response_triples']):
        if word in entity_vocab:
            flag = True
        if idx != -1: 
            assert word in entity_full_vocab, word
#     assert flag is True, flag 
    
    flag = False
    for word,idx in zip(my_case['post'],my_case['post_triples']):     
        if word in entity_vocab:
            flag = True
        if idx != 0:
            assert word in entity_full_vocab , word
#     assert flag is True

print('Good!')

#Word:30000  #Entity:27066
Good!


## Create & Load Vocabs

In [15]:
import json
valid_vocab = set()
with open(target_folder + 'uni_vocab.txt','w+',encoding='utf-8') as funi:
    valid_vocab = set()
    with open(ccm_path +'resource.txt') as f:
        d = json.loads(f.readline())
        csk_triples = d['csk_triples']
        csk_entities = d['csk_entities']
        raw_vocab = d['vocab_dict']
        with open(target_folder + 'vocab.txt','w+',encoding='utf-8') as fsrc:
                fsrc.write('<unk>\n<s>\n</s>\n')
                funi.write('<unk>\n<s>\n</s>\n')
                for word in  sorted(raw_vocab, key=raw_vocab.get, reverse=True)[0:vocab_size]:
                    fsrc.write('%s\n' % word)
                    funi.write('%s\n' % word)
                    valid_vocab.add(word)
    # Entity Vocab
    with open(ccm_path +'entity.txt',encoding='utf-8') as f:
        entity_list = []
        for idx, line in enumerate(f):
            entity = line.strip('\n')
            entity_list.append(entity)
        with open(target_folder + 'entity_vocab.txt','w+',encoding='utf-8') as fsrc:
                fsrc.write('#UNK\n#N\n#PE\n#NH\n#NT\n')
                funi.write('#UNK\n#N\n#PE\n#NH\n#NT\n')
                for word in  entity_list:
                    fsrc.write('%s\n' % word)
                    if word not in valid_vocab:
                        valid_vocab.add(word)
                        funi.write('%s\n' % word)

    # Relation Vocab
    with open(ccm_path +'relation.txt',encoding='utf-8') as f:
        entity_list = []
        for idx, line in enumerate(f):
            entity = line.strip('\n')
            entity_list.append(entity)
        with open(target_folder + 'relation_vocab.txt','w+',encoding='utf-8') as fsrc:
                fsrc.write('#NF\n#PR\n#NR\n')
                for word in  entity_list:
                    fsrc.write('%s\n' % word)


## Create Dialogue Query-Response Pairs

In [16]:
from collections import defaultdict
length_counter = defaultdict(int)
# Word_Level
for key in file_map:
    source_path = file_map[key]
    with open(ccm_path +source_path) as f:
        with open(target_folder+key+'.src','w+',encoding='utf-8') as fsrc:
             with open(target_folder+key+'.tgt','w+',encoding='utf-8') as ftgt:
                    for fidx, line in enumerate(f):
                        if key == 'demo' and fidx > 999:
                            break
                        item = json.loads(line)
                        message = item['post']
                        response = item['response']
                        fsrc.write('%s\n' % ' '.join(message))
                        ftgt.write('%s\n' % ' '.join(response))
                        length_counter[len(message)] += 1
                        length_counter[len(response)] += 1

## Generate Others

In [4]:
from collections import defaultdict
import random
random.seed(12345)
fact_dict = dict()
reverse_fact_dict = dict()
prefix = 'naf'
with open('stopwords.txt') as fin:
    stopwords = dict()
    for line in fin.readlines():
        word = line.strip('\n')
        stopwords[word] = 1
        
with open(target_folder+'uni_vocab.txt',encoding='utf-8') as funi:
    # Mask top 500
    for i in range(500):
        word = funi.readline().strip('\n')
        stopwords[word] = 2
        
with open(ccm_path +'resource.txt') as f:
    d = json.loads(f.readline())
    csk_triples = d['csk_triples']
    csk_entities = d['csk_entities']
    raw_vocab = d['vocab_dict']

max_golden_len = 0
max_fact_token_num = 0


for key in file_map:
    source_path = file_map[key]
    with open(ccm_path +source_path) as f:
        with open(target_folder+key+'.%s_fact' % prefix,'w+',encoding='utf-8') as fsrc:
            with open(target_folder+key+'.%s_golden_fact' % prefix,'w+',encoding='utf-8') as fgd:
                 with open(target_folder+key+'.%s_golden_facts' % prefix,'w+',encoding='utf-8') as fgds:
                    with open(target_folder+key+'.tgt.%s_ent' % prefix,'w+',encoding='utf-8') as fent:
                         with open(target_folder+key+'.tgt.%s_ment' % prefix,'w+',encoding='utf-8') as fment:
                            with open(target_folder+key+'.tgt.%s_cp' % prefix,'w+',encoding='utf-8') as fcp:
                                with open(target_folder+key+'.tgt.%s_entcp' % prefix,'w+',encoding='utf-8') as fec:
                                    for fidx, line in enumerate(f):
                                        
                                        if key == 'demo' and fidx > 2999:
                                            break
                                            
                                        my_case = json.loads(line)
                                        golden_et = [] # golden entities
                                        et = [] #all entities
                                        
                                        golden_triplet = set()
                                        for word,idx in zip(my_case['response'],my_case['response_triples']):
                                            if idx != -1:
                                                golden_triplet.add(idx)
                                                
                                        # NOT A FACT
                                        fact = ['#N','#N','#NH','#NF','#NT']
                                        ef = ' '.join(fact)
                                        if ef not in fact_dict:
                                            fact_dict[ef] = len(fact_dict)
                                            ef_id = fact_dict[ef]
                                            reverse_fact_dict[ef_id] = ef
                                        ef_id = str(fact_dict[ef])
                                        et.append(ef_id)

                                        position_dict = dict()

                                        for entity,triple in zip(my_case['all_entities'], my_case['all_triples']):
                                            for x,y in zip(entity,triple):
                                                ent = '%s' % csk_entities[x]
                                                fact = csk_triples[y].split(',')
                                                for i in range(3):
                                                    fact[i] = fact[i].strip(' ')
                                                entity_in_post = fact[0] if fact[0] != ent else fact[2]
                                                ef = ' '.join([entity_in_post, ent]+fact)
                                                if ef not in fact_dict:
                                                    fact_dict[ef] = len(fact_dict)
                                                    ef_id = fact_dict[ef]
                                                    reverse_fact_dict[ef_id] = ef
                                                ef_id = str(fact_dict[ef])
                                                if y in golden_triplet:
                                                    golden_et.append(ef_id)
                                                position_dict[y] = len(et)
                                                et.append(ef_id)
                              
                                        ent_response = []
                                        cp_response = []
                                        entcp_response = []
                                        copy_position_dict = dict()
                                        for i, post_word in enumerate(my_case['post']):
                                            if i > 49:
                                                break
                                            if post_word not in stopwords:
                                                copy_position_dict[post_word] = i

                                        for word,idx in zip(my_case['response'],my_case['response_triples']):
                                            if idx != -1 and idx in position_dict:
                                                entcp_response.append('$ENT_%d' % (position_dict[idx]))
                                                ent_response.append('$ENT_%d' % (position_dict[idx]))
                                                if max(max_fact_token_num, position_dict[idx]) > max_fact_token_num:
                                                    max_fact_token_num = max(max_fact_token_num, position_dict[idx]) 
                                                    print('max_fact_num %d' % max_fact_token_num)
                                                #ent_response.append('$ENT_%d_[%s]' % (position_dict[idx],word+'/'+reverse_fact_dict[int(et[position_dict[idx]])] ))
                                            elif word in copy_position_dict:
                                                entcp_response.append('$CP_%d' % (copy_position_dict[word]))
                                                ent_response.append(word)
                                            else:
                                                entcp_response.append(word)
                                                ent_response.append(word)

                                            if word in copy_position_dict:
                                                cp_response.append('$CP_%d' % (copy_position_dict[word])) 
                                            else:
                                                cp_response.append(word)
                                        fent.write('%s\n' % ' '.join(ent_response))
                                        fcp.write('%s\n' % ' '.join(cp_response))
                                        fec.write('%s\n' % ' '.join(entcp_response))
                                        fsrc.write('%s\n' % ' '.join(et))
                                        if len(golden_et) == 0:
                                            golden_et.append(str(0))
                                        fgds.write('%s\n' % ' '.join(golden_et))
                                        golden_et = random.sample(golden_et, 1)
                                        fgd.write('%s\n' % ' '.join(golden_et))


NameError: name 'target_folder' is not defined

In [18]:
with open(target_folder+'entity_vocab.txt',encoding='utf-8') as f:
    entity_words = [x.strip() for x in f.readlines()]

entity_dict = dict()
for entity in entity_words:
    entity_dict[entity] = len(entity_dict)

with open(target_folder+'vocab.txt',encoding='utf-8') as f:
    vocabs = [x.strip() for x in f.readlines()]

vocab_dict = dict(zip(vocabs, range(len(vocabs))))

with open(target_folder+'word2entity.txt','w+',encoding='utf-8') as f:
    for word in vocabs:
        if word not in entity_dict:
            f.write('0\n')
        else:
            f.write('%d\n' % entity_dict[word])
            
with open(target_folder+'entity2word.txt','w+',encoding='utf-8') as f:
    for entity in entity_words:
        if entity not in vocab_dict:
            f.write('0\n')
        else:
            f.write('%d\n' % vocab_dict[entity])


with open(target_folder+'%s_cpent_vocab.txt' % prefix,'w+',encoding='utf-8') as f:
    for word in vocabs:
        f.write('%s\n' % word)
    for i in range(35):
        f.write('$CP_%d\n' % i)
    for i in range(305):
        f.write('$ENT_%d\n' % i)



In [3]:
print(reverse_fact_dict[1])
with open(target_folder+'%s_fact_vocab.txt' % prefix,'w+', encoding='utf-8') as f:
    for i in range(len(reverse_fact_dict)):
        items = reverse_fact_dict[i].split()
        if i == 0 or (items[0] == items[2] and items[1] == items[4]):
            f.write('%s\n' % (reverse_fact_dict[i]))
        else:
            items[2] = items[0]
            items[4] = items[1]
            items[3] = items[3]
            f.write('%s\n' % (' '.join(items)))
fact_dict =None

NameError: name 'reverse_fact_dict' is not defined

In [2]:
fact_vocab = []
with open(target_folder+'%s_fact_vocab.txt' % prefix, encoding='utf-8') as f:
    for i, line in enumerate(f.readlines()):
        tgt_word = line.split()[1]
        fact_vocab.append(tgt_word)

print(fact_vocab[0:100])

for fact_vocab_size in [0]:
    tgt_prefx = prefix
    print(tgt_prefx)
    for key in file_map:
        source_path = file_map[key]
        with open(ccm_path+source_path) as f:
            with open(target_folder+key+'.%s_fact' % tgt_prefx,encoding='utf-8') as fsrc:
                with open(target_folder+key+'.%s_golden_fact' % tgt_prefx,encoding='utf-8') as fgd:
                     with open(target_folder+key+'.%s_golden_fact_position' % tgt_prefx, 'w+', encoding='utf-8') as frgd:
                            with open(target_folder+key+'.%s_neg_fact_position' % tgt_prefx, 'w+', encoding='utf-8') as fneg:
                                facts = [ x.strip('\n').split() for x in fsrc.readlines()]
                                golden_fact = [x.strip('\n') for x in fgd.readlines()]
                                for fact, golden in zip(facts, golden_fact):
                                    golden_index = fact.index(golden)
                                    neg_index = []
                                    for x in range(len(fact)):
                                        if fact_vocab[x] != fact_vocab[golden_index]: 
                                            neg_index.append(str(x))
    #                                     else:
    #                                         neg_index.append("XXXX"+str(x))
                                    frgd.write('%s\n' % golden_index)
                                    fneg.write('%s\n' % ' '.join(neg_index))


NameError: name 'target_folder' is not defined

In [1]:

prefix = 'naf'
fact_vocab = []
with open(target_folder+'%s_fact_vocab.txt' % prefix, encoding='utf-8') as f:
    for i, line in enumerate(f.readlines()):
        tgt_word = line.split()[1]
        fact_vocab.append(tgt_word)

print(fact_vocab[0:100])
for fact_vocab_size in [-1]:
    tgt_prefx = prefix
    print(tgt_prefx)
    for key in file_map:
        source_path = file_map[key]
        with open(ccm_path + source_path) as f:
            with open(target_folder+key+'.%s_fact' % tgt_prefx,encoding='utf-8') as fsrc:
                with open(target_folder+key+'.%s_golden_facts' % tgt_prefx,encoding='utf-8') as fgd:
                    with open(target_folder+key+'.%s_golden_facts_position' % tgt_prefx, 'w+', encoding='utf-8') as frgd:
                            facts = [ x.strip('\n').split() for x in fsrc.readlines()]
                            golden_facts = [x.strip('\n').split() for x in fgd.readlines()]
                            for fact, golden in zip(facts, golden_facts):
                                golden_index = set([str(fact.index(x)) for x in golden])
                                golden_index = ' '.join(golden_index)
                                frgd.write('%s\n' % golden_index)

NameError: name 'target_folder' is not defined